In [31]:
import streamlit as st
import pandas as pd
import requests
from datetime import datetime
from io import StringIO

In [ ]:
# Page config
st.set_page_config(page_title="Czech Football Clubs ELO", layout="wide")


In [32]:
czech_clubs = [
    'SlaviaPraha',
    'SpartaPraha',
    'ViktoriaPlzen',
    'BanikOstrava',
    'Jablonec',
    'HradecKralove',
    'SlovanLiberec',
    'SigmaOlomouc',
    'Slovacko',
    'CeskeBudejovice',
    'Teplice',
    'BohemiansPraha',
    'Karvina',
    'Dukla',
    'Pardubice',
    'MladaBoleslav'
]

In [33]:
def get_elo_data(club):
    try:
        """Fetch ELO data from the API"""
        response = requests.get(f'http://api.clubelo.com/{club}')
        df = pd.read_csv(StringIO(response.text))
        latest_elo = df.iloc[-1]['Elo']
        latest_date = df.iloc[-1]['From']
        return latest_elo, latest_date, df
    except:
        return None, None, None

In [34]:
# Create a list to store club data
clubs_data = []

# Fetch data for each club
for club in czech_clubs:
    elo, date, club_elo_df = get_elo_data(club)
    if elo is not None:
        # Format club name for display (remove camel case)
        display_name = ''.join([' ' + c if c.isupper() else c for c in club]).strip()
        clubs_data.append({
            'Club': display_name,
            'ELO Rating': round(elo),
            'Last Updated': date
        })

In [ ]:
import http.client

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "XxXxXxXxXxXxXxXxXxXxXxXx"
    }

conn.request("GET", "/fixtures?live=all", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))


In [35]:
# Convert to DataFrame and sort by ELO
clubs_df = pd.DataFrame(clubs_data)
clubs_df = clubs_df.sort_values('ELO Rating', ascending=False)


In [36]:

# App title
st.title("Czech Football Clubs ELO Ratings")

# Display stats
st.subheader("Current ELO Ratings")
st.dataframe(
    clubs_df,
    hide_index=True,
    column_config={
        'ELO Rating': st.column_config.NumberColumn(format="%d"),
        'Last Updated': st.column_config.DateColumn('Last Updated')
    }
)

# Calculate and display some statistics
st.subheader("League Statistics")
col1, col2, col3 = st.columns(3)

with col1:
    st.metric("Highest ELO", f"{clubs_df['ELO Rating'].max()}")
    st.caption(f"({clubs_df.iloc[0]['Club']})")
    
with col2:
    st.metric("Average ELO", f"{round(clubs_df['ELO Rating'].mean())}")
    
with col3:
    st.metric("Lowest ELO", f"{clubs_df['ELO Rating'].min()}")
    st.caption(f"({clubs_df.iloc[-1]['Club']})")

# Create a bar chart
st.subheader("ELO Ratings Comparison")
chart_data = clubs_df.set_index('Club')
st.bar_chart(chart_data['ELO Rating'])

2025-02-13 19:45:34.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.672 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [59]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def get_club_ids(season):
    clubs = pd.DataFrame(columns=["club_id", "club_name"])
    url = f"https://www.transfermarkt.com/1-fc-slovacko/spielplandatum/verein/5544/plus/0?saison_id={season}&wettbewerb_id=TS1"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to retrieve data")
        exit()

    soup = BeautifulSoup(response.text, 'html.parser')

    soup = soup.find("div", class_="responsive-table")
    # Finding match rows in the fixture table
    match_rows = soup.select("tr")

    for row in match_rows:
        zentriert_columns = [td.text.strip() for td in row.find_all("td", class_=["zentriert", "no-border-links hauptlink"])]
        if not zentriert_columns:
            continue
        club_id = re.search(r"/verein/(\d+)/", [td.find("a")["href"] for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")][0]).group(1)
        club_name = re.search(r"/([^/]+)/spielplan/", [td.find("a")["href"] for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")][0]).group(1)
        new_row = pd.DataFrame({"club_id": [club_id], "club_name": [club_name]})
        clubs = pd.concat([clubs, new_row], ignore_index=True)

    clubs = clubs[['club_id', 'club_name']].drop_duplicates()
    return clubs

clubs = get_club_ids(2024)
fixtures = []

for index, row in clubs.iterrows():
    club_id = row["club_id"]
    club_name = row["club_name"]
    url = f"https://www.transfermarkt.de/{club_name}/spielplandatum/verein/{club_id}/plus/0?saison_id=2024&wettbewerb_id=TS1"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to retrieve data")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    

    soup = soup.find("div", class_="responsive-table")
    # Finding match rows in the fixture table
    match_rows = soup.select("tr")

    for row in match_rows:
        zentriert_columns = [td.text.strip() for td in row.find_all("td", class_=["zentriert", "no-border-links hauptlink"])]
        if not zentriert_columns:
            continue
        
        if zentriert_columns[3] == 'H':
            home_team = club_name
            away_team = re.search(r"/([^/]+)/spielplan/", [td.find("a")["href"] for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")][0]).group(1)
        
        else:
            home_team = re.search(r"/([^/]+)/spielplan/", [td.find("a")["href"] for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")][0]).group(1)
            away_team = club_name

        event_date = zentriert_columns[1]
        event_time = zentriert_columns[2]

        fixtures.append({"home_team": home_team, "away_team": away_team, "event_date": str(event_date), "event_time": event_time})
    

fixtures = pd.DataFrame(fixtures)



In [61]:
fixtures.drop_duplicates()


,home_team,away_team,event_date,event_time
0,1-fc-slovacko,sk-slavia-prag,So. 21.07.24,17:00
1,sk-slavia-prag,sk-dynamo-ceske-budejovice,Sa. 27.07.24,20:00
2,fc-slovan-liberec,sk-slavia-prag,Fr. 02.08.24,17:30
3,sk-slavia-prag,sk-sigma-olmutz,Sa. 10.08.24,20:00
4,sk-slavia-prag,fk-teplice,Sa. 17.08.24,17:00
...,...,...,...,...
404,sk-sigma-olmutz,1-fc-slovacko,Mi. 27.11.24,17:00
412,fk-dukla-prag,sk-sigma-olmutz,Sa. 22.02.25,00:00
418,1-fc-slovacko,sk-sigma-olmutz,Sa. 12.04.25,00:00
436,fk-dukla-prag,1-fc-slovacko,Mi. 04.12.24,17:30


In [111]:
club_link = [td.find("a")["href"] for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")]

In [131]:
re.search(r"/([^/]+)/spielplan/", [td.find("a")["href"] for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")][0]).group(1)


'sk-slavia-prag'

In [125]:
re.search(r"/verein/(\d+)/", club_link[0]).group(1)

'62'

In [ ]:

    zentriert_columns = [td.text.strip() for td in row.find_all("td", class_=["zentriert", "no-border-links hauptlink"])]
    event_date = zentriert_columns[1] if len(zentriert_columns) > 1 else None
    club_link = [td.find("a")["href"] for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")]
    club_name = [td.find("a").text.strip() for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")]
    match = re.search(r"/verein/(\d+)/", club_link)
    club_id = match.group(1) if match else "N/A"
    print(club_id)


In [48]:
import requests
from bs4 import BeautifulSoup

def get_fixtures(team_name, verein_id):
    url = f"https://www.transfermarkt.com/{team_name}/spielplandatum/verein/{verein_id}/plus/0?saison_id=2024&wettbewerb_id=TS1"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to retrieve data")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    fixtures = []
    
    # Finding match rows in the fixture table
    match_rows = soup.select(".items tbody tr")
    
    for row in match_rows:
        row_style = row.get("style", "No style")
        columns = row.find_all("td")
        td_contents = [(td.text.strip(), td.get("class", [])) for td in columns]
        
        fixtures.append({"row_style": row_style, "td_contents": td_contents})
    
    return fixtures


# Example usage
team_name = "1-fc-slovacko"
verein_id = "5544"
fixtures = get_fixtures(team_name, verein_id)
for fixture in fixtures:
    print(fixture)


In [49]:
fixtures




[]